<a href="https://colab.research.google.com/github/financieras/campus/blob/main/pilas/pilas15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cálculo de ```s``` óptimo previsto para cada ```n```
* Consideremos pilas de ```n = 100``` números
* Generaremos 200 tiradas aleatoriamente
* Para cada tirada calculamos los pasos necesarios por el procedimiento PRIMERO
* Para el procedimiento SEGUNDO probaremos un rango de valores de ```s```, por ejemplo, entre 2 y 30.
* Para cada valor de ```s``` calculamos los pasos necesarios por el procedimiento SEGUNDO.
* Calculamos el valor de ```s``` que consigue el número menor de pasos y lo anotamos en la lista ```s_optimos```
* La lista ```s_optimos``` será finalmente una lista de 200 valores de ```s```
* De la lista ```s_optimos``` calculamos los valores:
 - media
 - round(media,0)
 - moda
 - mínimo
 - máximo

* De esta forma tendremos el valor recomendado de ```s``` para ```n=100```.
* Luego se prepite el procedimiento para diferentes valores de ```n```.
* Se monta la nube de puntos donde tenemos para cada ```n``` el valor de ```s``` aconsejado
* Se hace un ajuste por regresión polinómica y se obtiene el polinomio.
* De esta forma para un cierto ```n``` que nos den, sabremos cuál es el valor de ```s``` aconsejado para que el procedimiento SEGUNDO efectúe sus cálculos obteniendo el número menor de operaciones necesarias, en términos estimados.

In [1]:
######   FUNCIONES   ######
from random import sample, seed
seed()

def lista_ordenada(lis, orden_inverso=False): # la función da True si ponemos lista_ordenada([4,3,2,1], orden_inverso=True)
    ordenada = False
    aux = lis[:]
    aux.sort(reverse=orden_inverso)
    if aux == lis:
        ordenada = True
    return ordenada

def sa(a,b):
    if len(a) > 1: a[0],a[1] = a[1],a[0]
    return a,b
def sb(a,b):
    if len(b) > 1: b[0],b[1] = b[1],b[0]
    return a,b
def ss(a,b):
    sa(a,b)
    sb(a,b)
    return a,b
def pa(a,b):
    if len(b) > 0:
        a.insert(0, b[0])
        b.pop(0)
    return a,b
def pb(a,b):
    if len(a) > 0:
        b.insert(0, a[0])
        a.pop(0)
    return a,b
def ra(a,b):
    if len(a) > 1: a.append(a.pop(0))
    return a,b
def rb(a,b):
    if len(b) > 1: b.append(b.pop(0))
    return a,b
def rr(a,b):
    ra(a,b)
    rb(a,b)
    return a,b
def rra(a,b):
    if len(a) > 1: a.insert(0, a.pop())
    return a,b
def rrb(a,b):
    if len(b) > 1: b.insert(0, b.pop())
    return a,b
def rrr(a,b):
    rra(a,b)
    rrb(a,b)
    return a,b

def separa(tramo, n, s, a, b):   # se llevan de la pila A a la B los números del tramo que toca
    contador = 0
    ancho = int(n/s)
    fin = tramo * ancho          # fin del tramo
    inicio = (tramo-1)*ancho+1   # inicio del tramo
    pasados = 0   # cuenta cuantos números se han pasado de la pila A a la B
    while pasados < fin-inicio+1: # recorremos los elementos de la plia A hasta que pasemos todos los números de este tramo
        if inicio <= a[0] <= fin:
            pb(a,b)
            contador += 1
            pasados += 1
        else:
            ra(a,b)
            contador += 1
    return a, b, contador

def ordena(tramo, n, s, a, b):   # Pasamos de B a A los del tramo que toca, comenzando por el nº mayor, que para el tramo 1 es 5.
    # ordena los números recientemente separados y llevados a B
    # el objetivo es que esos números queden en B ordenados de forma decreciente: 5,4,3,2,1
    contador = 0
    ancho = int(n/s)
    fin = tramo * ancho          # fin del tramo
    inicio = (tramo-1)*ancho+1   # inicio del tramo
    while not lista_ordenada(b, orden_inverso=True): # se repite hasta que B quede ordenada decrecientemente. Si solo queda un elemento en B es que está ordenada
        while max(b) != b[0]:   # buscamos el máximo y vamos haciendo ra o rra hasta que quede el primero
            if b.index(max(b)) <= int(len(b)/2):
                rb(a,b)
            else:
                rrb(a,b)
            contador += 1
        if not lista_ordenada(b, orden_inverso=True):
            pa(a,b); contador += 1
    while len(a) > n-fin:   # llevamos de A a B los números del tramo que toca
        pb(a,b); contador += 1
    return a, b, contador

def ordena_final(tramo, n, s, a, b):    # ordena los números del úlitmo tramo que están en "A" y el objetivo es que queden crecientes: ...,18,19,20.
    contador = 0
    ancho = int(n/s)
    fin = n           # en el último tramo se ajusta para que el último nº sea igual a n
    inicio = (tramo-1)*ancho+1   # inicio del tramo
    while not lista_ordenada(a): # se repite hasta que A quede ordenada en orden creciente. Si solo queda un elemento en A es que está ordenada
        while min(a) != a[0]:
            if a.index(min(a)) <= int(len(a)/2):
                ra(a,b)
            else:
                rra(a,b)
            contador += 1
        if not lista_ordenada(a):
            pb(a,b); contador += 1
    while len(b) > 0:   # llevamos de B a A todos los números que hay en B
        pa(a,b); contador += 1
    return a, b, contador

In [17]:
for n in range(100,101,10):
    s_optimo = []  # lista que recoge el s que optimo para cada una de las tiradas que hagamos
    lista_porcentaje_mejora = []    # lista con el porcentaje de mejora del procedimiento SEGUNDO sobre el PRIMERO 
    suma_primeros = 0
    suma_segundos_optimos = 0
    num_tiradas = 200   # número de tiradas que se hacen, por ejemplo 200 tiradas para cada n, para ver luego la media, moda,...
    for tirada in range(num_tiradas): # nº de pilas A generadas pora cada n, cada una se llama "tirada"
        #mejor_primero = 0   # inicializamos un contador que dice cuantas veces es mejor el procedimiento PRIMERO  que el SEGUNDO
        # Generación de la pila A
        #n = 500   # número de elementos de la pila
        a = sample(range(1, n+1), n)
        a_original = a[:]
        b = []
        ######   PROCEDIMIENTO PRIMERO   ######
        contador = 0
        while not lista_ordenada(a):   # se repite hasta que A quede ordenada. Si solo queda un elemento en A es que está ordenada
            while min(a) != a[0]:   # buscamos el mínimo y vamos haciendo ra o rra hasta que quede el primero
                if a.index(min(a)) <= int(len(a)/2):
                    ra(a,b)
                else:
                    rra(a,b)
                contador += 1
            if not lista_ordenada(a):
                pb(a,b); contador += 1 #print(a,b) # luego hacemos pb
        while len(b) > 0:   # luego se hace pa pa pa pa hasta que no quede nadie en la pila b
            pa(a,b); contador += 1
        primero = contador   # la variable primero recoge el número de instrucciones necesarias para resolver esta tirada por el procedimiento PRIMERO
        suma_primeros += primero   # se acumulan todos los pasos de los 200 (num_tiradas) que se han realizado para luego calcular la media

        #### procedimiento SEGUNDO para varios valores de s en un cierto rango
        #for s in range(max(2,int(-0.000003*n**2+0.0155*n+1.6253)-1), int(-0.000002*n**2+0.01544*n+6.8537)+3):
        lista_pasos = [0,0]   # en esta lista recogemos cuántos pasos (ordenes) se necesitan por el procedimiento SEGUNDO para cada s, ponemos dos cero pq s comienz en 2
        for s in range(2,31):
            #print("s:", s, end=" ")
            ######   PROCEDIMIENTO SEGUNDO   ######
            a = a_original[:]
            n = len(a)
            b = []
            #s = 20   # número de grupos
            contador = 0
            for tramo in range(1, s):   # tramo=1,2,...,s-1. El último tramo se trata de forma diferente
                a, b, contador_separa = separa(tramo,n,s,a,b)   # lleva de A a B los números correspondientes a ese tramo
                contador += contador_separa
                a, b, contador_ordena = ordena(tramo,n,s,a,b)   # ordena los números recientemente separados y llevados a B
                contador += contador_ordena
            # tramo final donde los números de A ya se ordenan en A, ya no se usa la función separa
            # la pila de trabajo es "A" y el objetivo es que sea creciente: ...,18,19,20.
            a, b, contador_ordena_final = ordena_final(tramo,n,s,a,b)
            contador += contador_ordena_final
            #print("Al final:", a, b)
            #print("PROCEDIMIENTO SEGUNDO contador: ", contador)
            segundo=contador
            lista_pasos.append(segundo)
        #print(lista_pasos)
        s_optimo.append(lista_pasos.index(min(lista_pasos[2:])))
        lista_porcentaje_mejora.append(1-segundo/primero)
        suma_segundos_optimos += min(lista_pasos[2:])
    print("n:", n)
    print("min_s:\t", min(s_optimo))
    print("max_s:\t", max(s_optimo))
    print("media_s:", int(sum(s_optimo)/len(s_optimo)))
    print("moda_s:\t", max(set(s_optimo), key=s_optimo.count))
    primero_medio = suma_primeros/num_tiradas
    print("primero medio:", round(primero_medio))                      # media de las 200 (num_tiradas) del procedimiento PRIMERO para el n en el que estemos
    segundo_optimo_medio = suma_segundos_optimos/num_tiradas
    print("segundo óptimo medio:", round(segundo_optimo_medio))
    print(f"mejora: {round(100*(1-segundo_optimo_medio/primero_medio),0)}%" )

n: 100
min_s:	 3
max_s:	 8
media_s: 5
moda_s:	 5
primero medio: 1465
segundo óptimo medio: 844
mejora: 42.0%



In [ ]:
n=500
max(2,int(-0.000003*n**2+0.0155*n+1.6253)-1)
int(-0.000002*n**2+0.01544*n+6.8537)+2

16

In [ ]:
print("n, min_s, max_s, moda_s, media_PRIMERO, media_optimo_SEGUNDO, Mejora")
for n in range(2000,10001,1000):
    s_optimo = []  # lista que recoge el s que es óptimo para cada una de las tiradas que hagamos
    lista_porcentaje_mejora = []    # lista con el porcentaje de mejora del procedimiento SEGUNDO sobre el PRIMERO 
    suma_primeros = 0
    suma_segundos_optimos = 0
    num_tiradas = 100   # número de tiradas que se hacen, por ejemplo 200 tiradas para cada n, para ver luego la media, moda,...
    for tirada in range(num_tiradas): # nº de pilas A generadas pora cada n, cada una se llama "tirada"
        #mejor_primero = 0   # inicializamos un contador que dice cuantas veces es mejor el procedimiento PRIMERO  que el SEGUNDO
        # Generación de la pila A
        while True:   # evita que se genere justo la lista a con los números ya ordenados
            a = sample(range(1, n+1), n)
            if not lista_ordenada(a):
                break
        a_original = a[:]
        b = []
        ######   PROCEDIMIENTO PRIMERO   ######
        contador = 0
        while not lista_ordenada(a):   # se repite hasta que A quede ordenada. Si solo queda un elemento en A es que está ordenada
            while min(a) != a[0]:   # buscamos el mínimo y vamos haciendo ra o rra hasta que quede el primero
                if a.index(min(a)) <= int(len(a)/2):
                    ra(a,b)
                else:
                    rra(a,b)
                contador += 1
            if not lista_ordenada(a):
                pb(a,b); contador += 1 #print(a,b) # luego hacemos pb
        while len(b) > 0:   # luego se hace pa pa pa pa hasta que no quede nadie en la pila b
            pa(a,b); contador += 1
        primero = contador   # la variable primero recoge el número de instrucciones necesarias para resolver esta tirada por el procedimiento PRIMERO
        suma_primeros += primero   # se acumulan todos los pasos de los 200 (num_tiradas) que se han realizado para luego calcular la media

        #### procedimiento SEGUNDO para varios valores de s en un cierto rango
        #for s in range(max(2,int(-0.000003*n**2+0.0155*n+1.6253)-1), int(-0.000002*n**2+0.01544*n+6.8537)+3):
        lista_pasos = [0,0]   # en esta lista recogemos cuántos pasos (ordenes) se necesitan por el procedimiento SEGUNDO para cada s, ponemos dos cero pq s comienz en 2
        for s in range(10, min(n+1,36)):
            #print("s:", s, end=" ")
            ######   PROCEDIMIENTO SEGUNDO   ######
            a = a_original[:]
            n = len(a)
            b = []
            contador = 0
            for tramo in range(1, s):   # tramo=1,2,...,s-1. El último tramo se trata de forma diferente
                a, b, contador_separa = separa(tramo,n,s,a,b)   # lleva de A a B los números correspondientes a ese tramo
                contador += contador_separa
                a, b, contador_ordena = ordena(tramo,n,s,a,b)   # ordena los números recientemente separados y llevados a B
                contador += contador_ordena
            # tramo final donde los números de A ya se ordenan en A, ya no se usa la función separa
            # la pila de trabajo es "A" y el objetivo es que sea creciente: ...,18,19,20.
            a, b, contador_ordena_final = ordena_final(tramo,n,s,a,b)
            contador += contador_ordena_final
            segundo=contador
            lista_pasos.append(segundo)
        s_optimo.append(lista_pasos.index(min(lista_pasos[2:])))
        lista_porcentaje_mejora.append(1-segundo/primero)
        suma_segundos_optimos += min(lista_pasos[2:])
    # n, min_s, max_s, moda_s, media_PRIMERO, media_optimo_SEGUNDO, Mejora
    primero_medio = suma_primeros/num_tiradas
    segundo_optimo_medio = suma_segundos_optimos/num_tiradas
    print(n, min(s_optimo), max(s_optimo), max(set(s_optimo), key=s_optimo.count), round(primero_medio), round(segundo_optimo_medio), str(int(round(100*(1-segundo_optimo_medio/primero_medio),0)))+"%")
    #print("media_s:", int(sum(s_optimo)/len(s_optimo)))